In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv("heart_clean_final.csv")
df2=df

In [4]:
indep_X=df2.drop('target',axis=1)
dep_Y=df2['target']

In [9]:
def featureImportance(indep_X,dep_Y,n):
    importances_list=[]
    log_model = LogisticRegression(solver='lbfgs',max_iter=2000)
    RF = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    #NB = GaussianNB()
    DT= DecisionTreeClassifier(criterion = 'gini', max_features='sqrt',splitter='best',random_state = 0)
    svc_model = SVC(kernel = 'linear', random_state = 0)
    #knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    feature_imp=[log_model,svc_model,RF,DT]
    for model in feature_imp:
        print(model)
        model.fit(indep_X,dep_Y)
        if hasattr(model, "coef_"):
            importances = model.coef_[0]
        elif hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
        else:
            continue
        # Get top n feature indices
        top_n_indices = np.argsort(importances)[-n:]
        # Extract top n columns from indep_X
        selected_features = indep_X.iloc[:, top_n_indices]
        importances_list.append(selected_features)
    return importances_list

In [11]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        return X_train, X_test, y_train, y_test

In [13]:
def cm_prediction(classifier,X_test,y_test):
    y_pred = classifier.predict(X_test)
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    from sklearn.metrics import accuracy_score 
    from sklearn.metrics import classification_report 
    Accuracy=accuracy_score(y_test, y_pred )
    report=classification_report(y_test, y_pred)
    from sklearn.metrics import f1_score
    f1=f1_score(y_test,y_pred,average='binary')
    return Accuracy,report,cm,f1

In [15]:
def logistic(X_train,y_train,X_test,y_test):      
    from sklearn.linear_model import LogisticRegression
    classifier = LogisticRegression(random_state = 0)
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return  classifier,Accuracy,report,cm,f1

In [17]:
def svm_linear(X_train,y_train,X_test,y_test):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'linear', random_state = 0)
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return  classifier,Accuracy,report,cm,f1

In [19]:
def svm_NL(X_train,y_train,X_test,y_test):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'rbf', random_state = 0)
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return  classifier,Accuracy,report,cm,f1

In [21]:
def Naive(X_train,y_train,X_test,y_test):       
    from sklearn.naive_bayes import GaussianNB
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return  classifier,Accuracy,report,cm,f1 

In [23]:
def knn(X_train,y_train,X_test,y_test):
    from sklearn.neighbors import KNeighborsClassifier
    classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return  classifier,Accuracy,report,cm,f1 

In [25]:
def Decision(X_train,y_train,X_test,y_test):
    from sklearn.tree import DecisionTreeClassifier
    classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return  classifier,Accuracy,report,cm,f1      


In [27]:
def random(X_train,y_train,X_test,y_test):
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return  classifier,Accuracy,report,cm,f1

In [29]:
def Xgboost(X_train,y_train,X_test,y_test):
    from xgboost import XGBClassifier
    classifier =XGBClassifier()
    classifier.fit(X_train, y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return classifier,Accuracy,report,cm,f1 
    

In [31]:
def AdaBoost(X_train,y_train,X_test,y_test):
    from sklearn.ensemble import AdaBoostClassifier
    classifier=AdaBoostClassifier()
    classifier.fit(X_train,y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return classifier,Accuracy,report,cm,f1  

In [33]:
def LightGBM(X_train,y_train,X_test,y_test):
    from lightgbm import LGBMClassifier
    classifier=LGBMClassifier(verbose=-1)
    classifier.fit(X_train,y_train)
    Accuracy,report,cm,f1=cm_prediction(classifier,X_test,y_test)
    return classifier,Accuracy,report,cm,f1

In [35]:
def feature_classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf,accxg,accada,acclgbm): 
    feature_dataframe=pd.DataFrame(index=['Logistic','SVC','Random','DecisionTree'],columns=['Logistic','SVMl','SVMnl','KNN','Naive','Decision','Random','XgBoost','AdaBoost','LightGBM'])
    for number,idex in enumerate(feature_dataframe.index):
        feature_dataframe.loc[idex,'Logistic']=acclog[number]
        feature_dataframe.loc[idex,'SVMl']=accsvml[number]
        feature_dataframe.loc[idex,'SVMnl']=accsvmnl[number]
        feature_dataframe.loc[idex,'KNN']=accknn[number]
        feature_dataframe.loc[idex,'Naive']=accnav[number]
        feature_dataframe.loc[idex,'Decision']=accdes[number]
        feature_dataframe.loc[idex,'Random']=accrf[number]
        feature_dataframe.loc[idex,'XgBoost']=accxg[number]
        feature_dataframe.loc[idex,'AdaBoost']=accada[number]
        feature_dataframe.loc[idex,'LightGBM']=acclgbm[number]
    return feature_dataframe

In [37]:
X_train,X_test,y_train,y_test=split_scalar(indep_X,dep_Y)

In [51]:
importances_list=featureImportance(indep_X,dep_Y,8)       
acclog=[]
accsvml=[]
accsvmnl=[]
accknn=[]
accnav=[]
accdes=[]
accrf=[]
accxg=[]
accada=[]
acclgbm=[]
for i in importances_list:   
    X_train, X_test, y_train, y_test=split_scalar(i,dep_Y)   
    classifier,Accuracy,report,cm,f1=logistic(X_train,y_train,X_test,y_test)
    acclog.append(Accuracy)
    
    classifier,Accuracy,report,cm,f1=svm_linear(X_train,y_train,X_test,y_test)  
    accsvml.append(Accuracy)
    
    classifier,Accuracy,report,cm,f1=svm_NL(X_train,y_train,X_test,y_test)  
    accsvmnl.append(Accuracy)
    
    classifier,Accuracy,report,cm,f1=knn(X_train,y_train,X_test,y_test)  
    accknn.append(Accuracy)
    
    classifier,Accuracy,report,cm,f1=Naive(X_train,y_train,X_test,y_test)  
    accnav.append(Accuracy)
    
    classifier,Accuracy,report,cm,f1=Decision(X_train,y_train,X_test,y_test)  
    accdes.append(Accuracy)
    
    classifier,Accuracy,report,cm,f1=random(X_train,y_train,X_test,y_test)  
    accrf.append(Accuracy)
    
    classifier,Accuracy,report,cm,f1=Xgboost(X_train,y_train,X_test,y_test)  
    accxg.append(Accuracy)

    classifier,Accuracy,report,cm,f1=AdaBoost(X_train,y_train,X_test,y_test)  
    accada.append(Accuracy)

    classifier,Accuracy,report,cm,f1=LightGBM(X_train,y_train,X_test,y_test)  
    acclgbm.append(Accuracy)   

LogisticRegression(max_iter=2000)
SVC(kernel='linear', random_state=0)
RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)
DecisionTreeClassifier(max_features='sqrt', random_state=0)


In [53]:
result=feature_classification(acclog,accsvml,accsvmnl,accknn,accnav,accdes,accrf,accxg,accada,acclgbm)
highlighted_result=result.style.highlight_max(axis=1,color="lightgreen")
highlighted_result

,Logistic,SVMl,SVMnl,KNN,Naive,Decision,Random,XgBoost,AdaBoost,LightGBM
Logistic,0.869565,0.891304,0.847826,0.782609,0.782609,0.543478,0.760870,0.630435,0.826087,0.652174
SVC,0.869565,0.891304,0.847826,0.782609,0.782609,0.500000,0.804348,0.630435,0.826087,0.652174
Random,0.891304,0.869565,0.847826,0.847826,0.913043,0.782609,0.891304,0.760870,0.869565,0.847826
DecisionTree,0.869565,0.847826,0.826087,0.869565,0.891304,0.804348,0.760870,0.847826,0.891304,0.847826


In [55]:
f1log=[]
f1svml=[]
f1svmnl=[]
f1knn=[]
f1nav=[]
f1des=[]
f1rf=[]
f1xg=[]
f1ada=[]
f1lgbm=[]
for i in importances_list:   
    X_train, X_test, y_train, y_test=split_scalar(i,dep_Y)   
    classifier,Accuracy,report,cm,f1=logistic(X_train,y_train,X_test,y_test)
    f1log.append(f1)
    
    classifier,Accuracy,report,cm,f1=svm_linear(X_train,y_train,X_test,y_test)  
    f1svml.append(f1)
        
    classifier,Accuracy,report,cm,f1=svm_NL(X_train,y_train,X_test,y_test)  
    f1svmnl.append(f1)
        
    classifier,Accuracy,report,cm,f1=knn(X_train,y_train,X_test,y_test)  
    f1knn.append(f1)
        
    classifier,Accuracy,report,cm,f1=Naive(X_train,y_train,X_test,y_test)  
    f1nav.append(f1)
        
    classifier,Accuracy,report,cm,f1=Decision(X_train,y_train,X_test,y_test)  
    f1des.append(f1)
        
    classifier,Accuracy,report,cm,f1=random(X_train,y_train,X_test,y_test)  
    f1rf.append(f1)
    
    classifier,Accuracy,report,cm,f1=Xgboost(X_train,y_train,X_test,y_test)  
    f1xg.append(f1)

    classifier,Accuracy,report,cm,f1=AdaBoost(X_train,y_train,X_test,y_test)  
    f1ada.append(f1)

    classifier,Accuracy,report,cm,f1=LightGBM(X_train,y_train,X_test,y_test)  
    f1lgbm.append(f1)




In [57]:
def feature_classification(f1log,f1svml,f1svmnl,f1knn,f1nav,f1des,f1rf,f1xg,f1ada,f1lgbm):
    dataframe=pd.DataFrame(index=['Logistic','SVC','Random','DecisionTree'],columns=['Logistic','SVMl','SVMnl','KNN','Naive','Decision','Random','XgBoost','AdaBoost','LightGBM'])
    for number,idex in enumerate(dataframe.index):
        dataframe.loc[idex,'Logistic']=f1log[number]
        dataframe.loc[idex,'SVMl']=f1svml[number]
        dataframe.loc[idex,'SVMnl']=f1svmnl[number]
        dataframe.loc[idex,'KNN']=f1knn[number]
        dataframe.loc[idex,'Naive']=f1nav[number]
        dataframe.loc[idex,'Decision']=f1des[number]
        dataframe.loc[idex,'Random']=f1rf[number]
        dataframe.loc[idex,'XgBoost']=f1xg[number]
        dataframe.loc[idex,'AdaBoost']=f1ada[number]
        dataframe.loc[idex,'LightGBM']=f1lgbm[number]
    return dataframe     

In [59]:
result=feature_classification(f1log,f1svml,f1svmnl,f1knn,f1nav,f1des,f1rf,f1xg,f1ada,f1lgbm)
highlighted_result=result.style.highlight_max(axis=1,color="lightgreen")
highlighted_result

,Logistic,SVMl,SVMnl,KNN,Naive,Decision,Random,XgBoost,AdaBoost,LightGBM
Logistic,0.916667,0.931507,0.906667,0.868421,0.852941,0.676923,0.840580,0.753623,0.888889,0.771429
SVC,0.916667,0.931507,0.906667,0.868421,0.852941,0.610169,0.873239,0.753623,0.888889,0.771429
Random,0.931507,0.921053,0.906667,0.904110,0.942857,0.857143,0.933333,0.853333,0.918919,0.909091
DecisionTree,0.916667,0.904110,0.894737,0.921053,0.929577,0.869565,0.849315,0.906667,0.933333,0.906667
